In [ ]:
from load_image import *


# Inference using T2I

In [ ]:
from stable_diffusion import *

In [ ]:
negative_prompt = "extra digit, fewer digits, cropped, worst quality, low quality, glitch, deformed, mutated, ugly, disfigured"

# replace prompt as required by reference image

prompt=["Beautiful, carpet, 4k picture, high quality, color gradient rainbow , make it dim , similar to input image",
       "pattern,orange,red,yellow,purple,blue,artistic,floral,lace,motifs",
        "design,abstract,rich,dynamic,swirls,deep,vivid,decorative,warm,complex",
        "intricate,colorful,ornate,paisley,floral,detailed,swirling,vibrant,textured,elegant",
        "motifs,artistic,pattern,deep,warm,vivid,intricate,ornate,colorful,dynamic"]


In [ ]:
steps=5 # keep steps same as number of prompts and the image you want to generate
for i in range(steps):
  gen_images = pipe(
    prompt=prompt[i],
    negative_prompt=negative_prompt,
    image=image,
    num_inference_steps=30,
    guidance_scale=7,
    adapter_conditioning_scale=0.9,
    adapter_conditioning_factor=1
  ).images[0]
  gen_images.save(f'images/out_carpet{i}.png')

# inference using IP adapter


In [ ]:
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image
import torch

pipeline = AutoPipelineForImage2Image.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16).to("cuda")
pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter_sdxl.bin")
pipeline.set_ip_adapter_scale(0.6)

In [ ]:
image = load_image("https://designcompetition.explorug.online/images/Border/2/16.webp")
ip_image = load_image("https://designcompetition.explorug.online/images/Border/2/17.webp")

generator = torch.Generator(device="cpu").manual_seed(4)
images_l=[]
for _ in range(5):
    images = pipeline(
        prompt="best quality, high quality",
        image=image,
        ip_adapter_image=ip_image,
        generator=generator,
        strength=0.6,
    ).images[0]
    images_l.append(images)



In [ ]:
images_l[4]

# infer using stable_diffusion with clip
CLIP interogator generates prompts for the input/reference image , that prompt is passed through runway/stable_diffusion to generate new images that resembles original image

In [ ]:
!pip install replicate

In [ ]:
!export REPLICATE_API_TOKEN=<your_api_token> # from https://replicate.com/pharmapsychotic/clip-interrogator/api

In [ ]:
input = {
    "image": "path_to_your_image",  # Corrected file extension
    "clip_model_name": "ViT-L/14"
}

# Run the model
try:
    output = replicate.run(
        "pharmapsychotic/clip-interrogator:8151e1c9f47e696fa316146a2e35812ccf79cfc9eba05b11c7f450155102af70",
        input=input
    )
    print(output)
except replicate.exceptions.ReplicateError as e:
    print(f"Error: {e}")

In [ ]:
# call stability api
from io import BytesIO
import IPython
import json
import os
from PIL import Image
import requests
import time

In [ ]:
#@title Connect to the Stability API

import getpass
# @markdown To get your API key visit https://platform.stability.ai/account/keys
STABILITY_KEY = getpass.getpass('<your_api_key>')


In [ ]:
#@title Define functions

def send_generation_request(
    host,
    params,
):
    headers = {
        "Accept": "image/*",
        "Authorization": f"Bearer {STABILITY_KEY}"
    }

    # Encode parameters
    files = {}
    image = params.pop("image", None)
    mask = params.pop("mask", None)
    if image is not None and image != '':
        files["image"] = open(image, 'rb')
    if mask is not None and mask != '':
        files["mask"] = open(mask, 'rb')
    if len(files)==0:
        files["none"] = ''

    # Send request
    print(f"Sending REST request to {host}...")
    response = requests.post(
        host,
        headers=headers,
        files=files,
        data=params
    )
    if not response.ok:
        raise Exception(f"HTTP {response.status_code}: {response.text}")

    return response


In [ ]:

image =" path_to_image" #@param {type:"string"}
prompt = output
negative_prompt = "cropped, worst quality, low quality, glitch, deformed, mutated, ugly, disfigured" #@param {type:"string"}
seed = 1337 #@param {type:"integer"}
output_format = "jpeg" #@param ["jpeg", "png"]
strength = 0.75 #@param {type:"slider", min:0.0, max: 1.0, step: 0.01}

host = f"https://api.stability.ai/v2beta/stable-image/generate/sd3"

params = {
    "image" : image,
    "prompt" : prompt,
    "negative_prompt" : negative_prompt,
    "strength" : strength,
    "seed" : seed,
    "output_format": output_format,
    "model" : "sd3.5-large",
    "mode" : "image-to-image"
}

response = send_generation_request(
    host,
    params
)

# Decode response
output_image = response.content
finish_reason = response.headers.get("finish-reason")
seed = response.headers.get("seed")

# Check for NSFW classification
if finish_reason == 'CONTENT_FILTERED':
    raise Warning("Generation failed NSFW classifier")

# Save and display result
generated = f"images/generated_{seed}.{output_format}"
with open(generated, "wb") as f:
    f.write(output_image)
print(f"Saved image {generated}")

output.no_vertical_scroll()
print("Result image:")
IPython.display.display(Image.open(generated))
